In [1]:
from __future__ import print_function, division

from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import keras.backend as K
from PIL import Image
import matplotlib.pyplot as plt
from keras.utils.generic_utils import Progbar
import sys

import numpy as np
import os
class BGAN():
    """Reference: https://wiseodd.github.io/techblog/2017/03/07/boundary-seeking-gan/"""
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss=self.boundary_loss, optimizer=optimizer)
    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        # model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def boundary_loss(self, y_true, y_pred):
        """
        Boundary seeking loss.
        Reference: https://wiseodd.github.io/techblog/2017/03/07/boundary-seeking-gan/
        """
        return 0.5 * K.mean((K.log(y_pred) - K.log(1 - y_pred))**2)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = cifar10.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        # X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        nb_batches = int(X_train.shape[0] / batch_size)
        global_step=0

        for epoch in range(epochs):
            # ---------------------
            #  Train Discriminator
            # ---------------------
            for index in range(nb_batches):
                global_step+=1
                # progress_bar.update(index)

                # get a batch of real images
                image_batch = X_train[index * batch_size:(index + 1) * batch_size]
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(image_batch, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d global_step: %d[D loss: %f, acc.: %.2f%%] [G loss: %f]"
                      % (epoch,global_step, d_loss[0], 100 * d_loss[1], g_loss))

                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    self.sample_images(epoch,global_step)


    def sample_images(self, epoch,global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        generated_images = self.generator.predict(noise)
        # Rescale images 0 - 1
        # generated_images = 0.5 * gen_imgs + 0.5

        generated_images = np.asarray((generated_images * 127.5 + 127.5).astype(np.uint8))

        def vis_square(data, padsize=1, padval=0):
            # force the number of filters to be square
            n = int(np.ceil(np.sqrt(data.shape[0])))
            padding = ((0, n ** 2 - data.shape[0]), (0, padsize), (0, padsize)) + ((0, 0),) * (data.ndim - 3)
            data = np.pad(data, padding, mode='constant', constant_values=(padval, padval))

            # tile the filters into an image
            data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
            data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
            return data

        img = vis_square(generated_images)
        if not os.path.isdir('images_bgan_cifar10'):
            os.mkdir('images_bgan_cifar10')
        Image.fromarray(img).save(
            "images_bgan_cifar10/epoch_%d_%d.png" % (epoch,global_step))


if __name__ == '__main__':
    bgan = BGAN()
    bgan.train(epochs=50, batch_size=64, sample_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 64)          0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 global_step: 1[D loss: 1.026567, acc.: 39.84%] [G loss: 0.615264]
epoch:0 global_step: 2[D loss: 0.915361, acc.: 49.22%] [G loss: 0.663395]
epoch:0 global_step: 3[D loss: 0.799096, acc.: 46.09%] [G loss: 0.771527]
epoch:0 global_step: 4[D loss: 0.745886, acc.: 53.12%] [G loss: 0.734093]
epoch:0 global_step: 5[D loss: 0.732165, acc.: 56.25%] [G loss: 0.914225]
epoch:0 global_step: 6[D loss: 0.781520, acc.: 53.12%] [G loss: 0.726156]
epoch:0 global_step: 7[D loss: 0.704415, acc.: 61.72%] [G loss: 0.706325]
epoch:0 global_step: 8[D loss: 0.699240, acc.: 60.16%] [G loss: 0.924473]
epoch:0 global_step: 9[D loss: 0.697456, acc.: 60.16%] [G loss: 1.030122]
epoch:0 global_step: 10[D loss: 0.763934, acc.: 50.78%] [G loss: 0.779138]
epoch:0 global_step: 11[D loss: 0.753963, acc.: 57.81%] [G loss: 0.943324]
epoch:0 global_step: 12[D loss: 0.842680, acc.: 47.66%] [G loss: 0.724519]
epoch:0 global_step: 13[D loss: 0.724346, acc.: 57.03%] [G loss: 0.885031]
epoch:0 global_step: 14[D loss: 0.

epoch:0 global_step: 111[D loss: 0.767215, acc.: 56.25%] [G loss: 1.378437]
epoch:0 global_step: 112[D loss: 0.755010, acc.: 60.94%] [G loss: 1.718548]
epoch:0 global_step: 113[D loss: 0.685715, acc.: 63.28%] [G loss: 1.655150]
epoch:0 global_step: 114[D loss: 0.781296, acc.: 58.59%] [G loss: 1.491717]
epoch:0 global_step: 115[D loss: 0.716198, acc.: 55.47%] [G loss: 1.814241]
epoch:0 global_step: 116[D loss: 0.827273, acc.: 51.56%] [G loss: 1.236284]
epoch:0 global_step: 117[D loss: 0.763687, acc.: 54.69%] [G loss: 1.560329]
epoch:0 global_step: 118[D loss: 0.560114, acc.: 74.22%] [G loss: 1.780988]
epoch:0 global_step: 119[D loss: 0.809756, acc.: 53.12%] [G loss: 1.229738]
epoch:0 global_step: 120[D loss: 0.591678, acc.: 65.62%] [G loss: 1.322445]
epoch:0 global_step: 121[D loss: 0.592702, acc.: 64.06%] [G loss: 1.361496]
epoch:0 global_step: 122[D loss: 0.552619, acc.: 71.09%] [G loss: 1.399065]
epoch:0 global_step: 123[D loss: 0.624307, acc.: 67.97%] [G loss: 1.086004]
epoch:0 glob

...

epoch:49 global_step: 38551[D loss: 0.762886, acc.: 50.00%] [G loss: 0.710580]
epoch:49 global_step: 38552[D loss: 0.497727, acc.: 75.78%] [G loss: 0.742652]
epoch:49 global_step: 38553[D loss: 0.605581, acc.: 69.53%] [G loss: 0.495053]
epoch:49 global_step: 38554[D loss: 0.498680, acc.: 75.78%] [G loss: 0.583475]
epoch:49 global_step: 38555[D loss: 0.612950, acc.: 67.19%] [G loss: 1.009739]
epoch:49 global_step: 38556[D loss: 0.640793, acc.: 64.84%] [G loss: 0.692709]
epoch:49 global_step: 38557[D loss: 0.735831, acc.: 52.34%] [G loss: 1.029749]
epoch:49 global_step: 38558[D loss: 0.708531, acc.: 57.81%] [G loss: 0.661261]
epoch:49 global_step: 38559[D loss: 0.665485, acc.: 60.16%] [G loss: 0.985182]
epoch:49 global_step: 38560[D loss: 0.392505, acc.: 84.38%] [G loss: 0.790856]
epoch:49 global_step: 38561[D loss: 0.912389, acc.: 39.84%] [G loss: 0.593677]
epoch:49 global_step: 38562[D loss: 0.549595, acc.: 75.00%] [G loss: 0.854897]
epoch:49 global_step: 38563[D loss: 0.725164, acc.: 

epoch:49 global_step: 38656[D loss: 0.410392, acc.: 85.94%] [G loss: 1.506195]
epoch:49 global_step: 38657[D loss: 0.730395, acc.: 54.69%] [G loss: 1.754659]
epoch:49 global_step: 38658[D loss: 0.841650, acc.: 41.41%] [G loss: 0.378607]
epoch:49 global_step: 38659[D loss: 0.623059, acc.: 62.50%] [G loss: 0.783227]
epoch:49 global_step: 38660[D loss: 0.523200, acc.: 72.66%] [G loss: 0.774791]
epoch:49 global_step: 38661[D loss: 0.543796, acc.: 75.78%] [G loss: 0.414842]
epoch:49 global_step: 38662[D loss: 0.427433, acc.: 89.84%] [G loss: 0.801770]
epoch:49 global_step: 38663[D loss: 0.392618, acc.: 88.28%] [G loss: 1.404581]
epoch:49 global_step: 38664[D loss: 0.720097, acc.: 53.12%] [G loss: 1.011626]
epoch:49 global_step: 38665[D loss: 0.466615, acc.: 78.12%] [G loss: 1.500394]
epoch:49 global_step: 38666[D loss: 0.740699, acc.: 50.78%] [G loss: 0.857109]
epoch:49 global_step: 38667[D loss: 0.642238, acc.: 66.41%] [G loss: 1.392393]
epoch:49 global_step: 38668[D loss: 0.485898, acc.: 

epoch:49 global_step: 38761[D loss: 0.431428, acc.: 90.62%] [G loss: 1.084255]
epoch:49 global_step: 38762[D loss: 0.580360, acc.: 69.53%] [G loss: 0.674425]
epoch:49 global_step: 38763[D loss: 0.700763, acc.: 50.00%] [G loss: 0.528708]
epoch:49 global_step: 38764[D loss: 0.526467, acc.: 74.22%] [G loss: 0.961781]
epoch:49 global_step: 38765[D loss: 0.724112, acc.: 57.81%] [G loss: 0.897644]
epoch:49 global_step: 38766[D loss: 0.588837, acc.: 67.97%] [G loss: 0.908408]
epoch:49 global_step: 38767[D loss: 0.603697, acc.: 65.62%] [G loss: 0.562979]
epoch:49 global_step: 38768[D loss: 0.348894, acc.: 89.06%] [G loss: 0.998046]
epoch:49 global_step: 38769[D loss: 0.658112, acc.: 66.41%] [G loss: 1.070011]
epoch:49 global_step: 38770[D loss: 0.526517, acc.: 71.09%] [G loss: 1.098844]
epoch:49 global_step: 38771[D loss: 0.491802, acc.: 81.25%] [G loss: 0.718387]
epoch:49 global_step: 38772[D loss: 0.582683, acc.: 70.31%] [G loss: 0.620326]
epoch:49 global_step: 38773[D loss: 0.648679, acc.: 

epoch:49 global_step: 38866[D loss: 0.644081, acc.: 58.59%] [G loss: 1.044838]
epoch:49 global_step: 38867[D loss: 0.598101, acc.: 64.84%] [G loss: 0.935246]
epoch:49 global_step: 38868[D loss: 0.674250, acc.: 60.94%] [G loss: 0.420480]
epoch:49 global_step: 38869[D loss: 0.391904, acc.: 84.38%] [G loss: 1.192623]
epoch:49 global_step: 38870[D loss: 0.744057, acc.: 51.56%] [G loss: 0.865575]
epoch:49 global_step: 38871[D loss: 0.654748, acc.: 60.16%] [G loss: 0.629301]
epoch:49 global_step: 38872[D loss: 0.560874, acc.: 74.22%] [G loss: 1.090319]
epoch:49 global_step: 38873[D loss: 0.666823, acc.: 58.59%] [G loss: 0.729320]
epoch:49 global_step: 38874[D loss: 0.522238, acc.: 71.88%] [G loss: 1.051600]
epoch:49 global_step: 38875[D loss: 0.575383, acc.: 71.09%] [G loss: 0.600083]
epoch:49 global_step: 38876[D loss: 0.493178, acc.: 79.69%] [G loss: 1.012868]
epoch:49 global_step: 38877[D loss: 0.671273, acc.: 60.16%] [G loss: 0.417618]
epoch:49 global_step: 38878[D loss: 1.047026, acc.: 

epoch:49 global_step: 38971[D loss: 0.754059, acc.: 54.69%] [G loss: 0.531392]
epoch:49 global_step: 38972[D loss: 0.514569, acc.: 71.09%] [G loss: 0.671407]
epoch:49 global_step: 38973[D loss: 0.611296, acc.: 69.53%] [G loss: 0.668251]
epoch:49 global_step: 38974[D loss: 0.515191, acc.: 72.66%] [G loss: 0.888903]
epoch:49 global_step: 38975[D loss: 0.502987, acc.: 77.34%] [G loss: 0.827547]
epoch:49 global_step: 38976[D loss: 0.732369, acc.: 52.34%] [G loss: 2.102351]
epoch:49 global_step: 38977[D loss: 0.587138, acc.: 70.31%] [G loss: 0.588766]
epoch:49 global_step: 38978[D loss: 0.448601, acc.: 79.69%] [G loss: 0.735742]
epoch:49 global_step: 38979[D loss: 0.463859, acc.: 78.91%] [G loss: 1.424639]
epoch:49 global_step: 38980[D loss: 0.499330, acc.: 78.91%] [G loss: 0.797613]
epoch:49 global_step: 38981[D loss: 0.761601, acc.: 50.78%] [G loss: 1.064287]
epoch:49 global_step: 38982[D loss: 0.451874, acc.: 80.47%] [G loss: 1.968273]
epoch:49 global_step: 38983[D loss: 0.547635, acc.: 